# Langchain - Basic Integration for LLMOps
* Notebook by Adam Lang
* Date: 12/23/2024

# Install Dependencies

* Make sure you have an `OpenAI` key saved as a .env file before running this notebook.

In [2]:
! pip install -r requirements.txt

  Using cached langchain_chroma-0.1.4-py3-none-any.whl.metadata (1.6 kB)
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
  Using cached tiktoken-0.8.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

# Load Environment Variables

In [3]:
import dotenv

# OPENAI_API_KEY=
dotenv.load_dotenv('.env')

True

# Prompt Setup
* First we need to set up the core prompts we will use for a basic LLM call to the Open AI API. 

In [4]:
from langchain_core.prompts import ChatPromptTemplate ## prompt templates

## There are 4 basic personas for the ChatPromptTemplate below
# 'human', 'user', 'ai', 'assistant', or 'system'
prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful AI bot. Your name is {name}."),
            ("human", "Hello, how are you doing today?"),
            ("ai", "I'm doing well, thanks!"),
            ("human", "{user_input}"),
        ])

## invoke final prompt 
## invoke or async_invoke are the usual invoke methods
final_prompt = prompt.invoke({
        "name": "Bob",
        "user_input": "What is your name?"
    })

## print final_prompt
final_prompt

ChatPromptValue(messages=[SystemMessage(content='You are a helpful AI bot. Your name is Bob.'), HumanMessage(content='Hello, how are you doing today?'), AIMessage(content="I'm doing well, thanks!"), HumanMessage(content='What is your name?')])

In [5]:
from langchain_core.prompts import MessagesPlaceholder ## define a place holder variable


## example using MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot."),
    MessagesPlaceholder(variable_name="conversation", optional=True)
])

# Conversation can be retireved from other sources like Vector store, other APIs
final_prompt = prompt.invoke(
    {
        "conversation": [
            ("human", "Hi!"),
            ("ai", "How can I assist you today?"),
            ("human", "Can you make me an ice cream sundae?"),
            ("ai", "No.")
        ]
    }
)

final_prompt

ChatPromptValue(messages=[SystemMessage(content='You are a helpful AI bot.'), HumanMessage(content='Hi!'), AIMessage(content='How can I assist you today?'), HumanMessage(content='Can you make me an ice cream sundae?'), AIMessage(content='No.')])

# Setup LLM via OpenAI API
* Setup connection to LLM.

In [6]:
from langchain_openai import ChatOpenAI

## init the LLM of choice
llm = ChatOpenAI(model="gpt-4o-mini")

## invoke the LLM with a zero shot question
llm.invoke("Hello, how are you doing?")

AIMessage(content="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 14, 'total_tokens': 44, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-830ae7ca-479c-4f00-acbc-47c534c5fe89-0', usage_metadata={'input_tokens': 14, 'output_tokens': 30, 'total_tokens': 44})

In [7]:
# using a different model
## init the new llm model
llm = ChatOpenAI(model="gpt-4o")

## invoke same zero shot input
llm.invoke("Hello, how are you doing?")

AIMessage(content="Hello! I'm here and ready to help you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 14, 'total_tokens': 32, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_9faba9f038', 'finish_reason': 'stop', 'logprobs': None}, id='run-db7b9a90-54c3-461b-afcb-059fd6df362c-0', usage_metadata={'input_tokens': 14, 'output_tokens': 18, 'total_tokens': 32})

In [8]:
# different Hyperparameters
## temp=0 is deterministic
print(ChatOpenAI(model="gpt-4o", temperature=0, max_tokens=100).invoke("Hello, how are you doing?"))
print("\n")
## temp=2 is probabilistic
print(ChatOpenAI(model="gpt-4o", temperature=2, max_tokens=100).invoke("Hello, how are you doing?"))

content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 14, 'total_tokens': 44, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_5f20662549', 'finish_reason': 'stop', 'logprobs': None} id='run-c95c5889-bd62-4cd6-b0a1-8c7539ac2faa-0' usage_metadata={'input_tokens': 14, 'output_tokens': 30, 'total_tokens': 44}


content="Hello! I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help you with whatever you need. How can I assist you today?" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 14, 

## Different Output Parser
* We can change the structured output of the model.

In [9]:
### Different Output Parser
from langchain_core.output_parsers import StrOutputParser
chain = llm | StrOutputParser()
chain.invoke("Hello, how are you doing?")

"Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"

# Chains
* This is the most basic use case of LangChain for LLMOps using chains.
* It is also one of the most common use case. 

In [16]:
## 1. Setup prompt
prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful AI bot. Your name is {name}."),
            ("human", "Hello, how are you doing?"),
            ("ai", "I'm doing well, thanks!"),
            ("human", "{user_input}"),
        ])
## 2. Setup LLM and parser
llm = ChatOpenAI(model="gpt-4o", temperature=0, max_tokens=100)
parser = StrOutputParser()

## 3. Create Chain using LCEL syntax
chain = (prompt 
         | 
        llm 
         | 
        parser)

## 4. invoke chain
chain.invoke({
        "name": "Bob",
        "user_input": "What is your name?"
})

'My name is Bob. How can I assist you today?'

## Streaming Example

In [17]:
## Streaming invocation
stream = chain.stream({
        "name": "Bob",
        "user_input": "What is your name?"
})


for token in stream:
    print(token)


My
 name
 is
 Bob
.
 How
 can
 I
 assist
 you
 today
?



# Saving and Loading chain
* What we see below is we can save the chain in a separate json file and load it at runtime. 

In [18]:
from langchain_core.load import dumpd
import json

json_str_chain = dumpd(chain)
with open("./chain.json", "w") as fp:
    json.dump(json_str_chain, fp, indent=4)

Note about below:
* The chain is smart enough to not save our API key secrets to the json file.

In [19]:
chain.middle[0].lc_secrets

{'openai_api_key': 'OPENAI_API_KEY'}

Now we can load the chain below from the json file.

In [20]:
from langchain_core.load import load
import os

with open("./chain.json", "r") as fp:
    chain = load(json.load(fp), secrets_map={"OPENAI_API_KEY": os.getenv("OPENAI_API_KEY")})
chain

ChatPromptTemplate(input_variables=['name', 'user_input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], template='You are a helpful AI bot. Your name is {name}.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Hello, how are you doing?')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="I'm doing well, thanks!")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], template='{user_input}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fa73d5f3b90>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fa73d95b5d0>, root_client=<openai.OpenAI object at 0x7fa73d42fa10>, root_async_client=<openai.AsyncOpenAI object at 0x7fa73d5f14d0>, model_name='gpt-4o', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=100)
| StrOutputParser()

Now we can invoke the chain.

In [21]:
chain.invoke({
        "name": "Bob",
        "user_input": "What is your name?"
})

'My name is Bob. How can I assist you today?'